## API REQUEST CALL AND DATA FILE CREATION

In [20]:
import requests
import json



fdic_data = requests.get("https://banks.data.fdic.gov/api/summary?filters=YEAR%3A%5B%222000%22%20TO%20%222019%22%5D&fields=STNAME%2CYEAR%2CINTINC%2CEINTEXP%2CNIM%2CNONII%2CNONIX%2CELNATR%2CITAXR%2CIGLSEC%2CITAX%2CEXTRA%2CNETINC&sort_by=YEAR&sort_order=DESC&limit=3000&offset=0&format=json&download=false&filename=data_file")
print(x.status_code)

data = fdic_data.json()
with open ('data.json', 'w') as fd:
    json.dump(data,fd)



200


## DROPING DATA INTO MONGODB

In [26]:

#!pip install pymongo
from pymongo import MongoClient

#Making Connection to MOngoDB
client = MongoClient("mongodb://localhost:27017/")

#Database
data_b = client["FDICDATABASE"]

#collection
Collection = data_b["fdic"]

#loading the json file
with open('data.json') as file:
    file_data = json.load(file)

#inserting the loaded file into the fdic collection
if isinstance(file_data, list):
    Collection.insert_many(file_data)
else:
    Collection.insert_one(file_data)
    
client.close()
    


## EXTRACTING DATA FROM MONGODB

In [3]:
#mongodb connection
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")
db = client['FDICDATABASE']
collection = db['fdic']
x = collection.find()
for fd_data in x:
    print(fd_data)
    

{'_id': ObjectId('61cca6a0d0db98efb09e350a'), 'meta': {'total': 2440, 'parameters': {'filters': 'YEAR:["2000" TO "2019"]', 'fields': 'STNAME,YEAR,INTINC,EINTEXP,NIM,NONII,NONIX,ELNATR,ITAXR,IGLSEC,ITAX,EXTRA,NETINC', 'limit': '3000', 'offset': '0'}, 'index': {'name': 'risview_aggregate_1639450652740', 'createTimestamp': '2021-12-14T02:57:32Z'}}, 'data': [{'data': {'ITAX': 0, 'EINTEXP': 0, 'EXTRA': 0, 'ELNATR': 0, 'STNAME': 'Palau', 'INTINC': 0, 'NETINC': 0, 'NONIX': 0, 'NIM': 0, 'YEAR': '2019', 'ITAXR': 0, 'IGLSEC': 0, 'NONII': 0, 'ID': 'CB_2019_'}, 'score': 0}, {'data': {'ITAX': 25103, 'EINTEXP': 23811, 'EXTRA': 0, 'ELNATR': 86, 'STNAME': 'Alaska', 'INTINC': 259875, 'NETINC': 85359, 'NONIX': 192575, 'NIM': 236064, 'YEAR': '2019', 'ITAXR': 110254, 'IGLSEC': 208, 'NONII': 66851, 'ID': 'CB_2019_AK'}, 'score': 0}, {'data': {'ITAX': 687996, 'EINTEXP': 1965891, 'EXTRA': 0, 'ELNATR': 1067032, 'STNAME': 'Alabama', 'INTINC': 9918405, 'NETINC': 2416555, 'NONIX': 6986693, 'NIM': 7952514, 'YEAR':

## CLEANING THE DATA

In [28]:
#normalizing the data
import pandas as pd
new_data = fd_data['data']
norm_data = pd.json_normalize(new_data)
#norm_data.head()

#checking for missing values
norm_data.isnull()


#drop the score column in the dataframe
fdic_data=norm_data.drop(['score'], axis =1)
fdic_data
print(fdic_data.columns)
#renaming my columns
fdic_data.columns = ['ITAX', 'EINTEXP', 'EXTRA', 'ELNATR', 'STNAME', 'INTINC', 'NETINC', 'NONIX', 'NIM', 'YEAR', 'ITAXR', 'IGLSEC', 'NONII', 'ID']
fdic_data.head()

print(fdic_data.dtypes)


Index(['data.ITAX', 'data.EINTEXP', 'data.EXTRA', 'data.ELNATR', 'data.STNAME',
       'data.INTINC', 'data.NETINC', 'data.NONIX', 'data.NIM', 'data.YEAR',
       'data.ITAXR', 'data.IGLSEC', 'data.NONII', 'data.ID'],
      dtype='object')
ITAX        int64
EINTEXP     int64
EXTRA       int64
ELNATR      int64
STNAME     object
INTINC      int64
NETINC      int64
NONIX       int64
NIM         int64
YEAR       object
ITAXR       int64
IGLSEC      int64
NONII       int64
ID         object
dtype: object


## CONVERTING INTO A CSV FILE FOR POSTGRESSQL

In [24]:
#conerting the data into csv
fdic_data.to_csv(r'C:\Users\mamma\Documents\fdic_dataframe.csv', index = False, header = True)
#print(fdic_data)

## CONNECTING AND DROPPING DATA INTO POSTGRESSQL

In [85]:
#!pip install psycopg2
import psycopg2 

#establish a connection to database
con = psycopg2.connect(database = "FDICDATABASE",
                      user = 'postgres',
                      password = '1234',
                      host ='127.0.0.1',
                      port = '5432')

con.autocommit = True
cursor = con.cursor()

#creating database 
psql = '''CREATE TABLE FDICDATABASE4(ITAX int, EINTEXP int, EXTRA int, ELNATR int, STNAME text, INTINC int, NETINC int, NONIX int, NIM int, YEAR char(20), ITAXR int, IGLSEC int, NONII int, ID varchar(30));'''

cursor.execute(psql)




In [86]:
#copying data from csv file
psql2 = '''COPY fdicdatabase4(ITAX, EINTEXP, EXTRA , ELNATR, STNAME, INTINC, NETINC, NONIX, NIM, YEAR, ITAXR, IGLSEC, NONII, ID)
FROM 'C:/temp/fdic_dataframe.csv'
DELIMITER ',' 
CSV HEADER;'''

cursor.execute(psql2)



In [87]:
psql3 = '''select * from fdicdatabase4'''
cursor.execute(psql3)
for i in cursor.fetchall():
    print(i)
  
con.commit()
con.close()

(0, 0, 0, 0, 'Palau', 0, 0, 0, 0, '2019                ', 0, 0, 0, 'CB_2019_')
(25103, 23811, 0, 86, 'Alaska', 259875, 85359, 192575, 236064, '2019                ', 110254, 208, 66851, 'CB_2019_AK')
(687996, 1965891, 0, 1067032, 'Alabama', 9918405, 2416555, 6986693, 7952514, '2019                ', 3082137, 24572, 3183348, 'CB_2019_AL')
(423205, 1028167, 0, 190356, 'Arkansas', 4996461, 1623727, 3179451, 3968294, '2019                ', 2001673, 45237, 1403186, 'CB_2019_AR')
(0, 0, 0, 0, 'American Samoa', 0, 0, 0, 0, '2019                ', 0, 0, 0, 'CB_2019_AS')
(118369, 196181, 0, 23665, 'Arizona', 1404749, 554984, 688637, 1208568, '2019                ', 666807, 6543, 170541, 'CB_2019_AZ')
(2827622, 6706771, -2015, 1073171, 'California', 30710072, 7474764, 20541263, 24003301, '2019                ', 10071631, 218938, 7682764, 'CB_2019_CA')
(168906, 301215, 6724, 43506, 'Colorado', 2265935, 778074, 1525184, 1964720, '2019                ', 934215, 6028, 538185, 'CB_2019_CO')
(252092,

(1145239, 3324355, -14, 189887, 'Michigan', 9840571, 2340235, 5524865, 6516216, '2005                ', 3386986, 98502, 2585522, 'CB_2005_MI')
(221587, 1074765, -225, 100492, 'Minnesota', 3554603, 980316, 2003273, 2479838, '2005                ', 1194190, 7938, 818117, 'CB_2005_MN')
(429135, 1612075, 4, 130310, 'Missouri', 4764557, 1091007, 2679314, 3152482, '2005                ', 1524116, -3978, 1181258, 'CB_2005_MO')
(213147, 766291, 694, 119015, 'Mississippi', 2355689, 501929, 1396978, 1589398, '2005                ', 717987, -3605, 644582, 'CB_2005_MS')
(77696, 227487, 0, 25597, 'Montana', 854940, 232896, 426300, 627453, '2005                ', 313833, -3241, 138277, 'CB_2005_MT')
(10990358, 29798913, 214000, 72788, 'North Carolina', 73873007, 22797014, 40113190, 44074094, '2005                ', 32540864, 1032508, 28652748, 'CB_2005_NC')
(105600, 330693, -82, 47749, 'North Dakota', 838433, 259331, 540744, 507740, '2005                ', 365147, -134, 445900, 'CB_2005_ND')
(123609